In [1]:
pip install cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 87.4 MB/s eta 0:00:00


In [2]:
import cohere

co = cohere.ClientV2(api_key="DuJ7RzyWW1iKXVIgeh6SI7j2xXTQgpiIoexIqbhp")

response = co.chat(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant"
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "who is trump"
                }
            ]
        }
    ],
    temperature=0.3,
    model="command-a-03-2025",
)


In [3]:
response

V2ChatResponse(id='9fd2d0fa-301b-4c7a-8a6e-72e6a1ee160b', finish_reason='COMPLETE', message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Donald John Trump is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021. Born on June 14, 1946, in Queens, New York City, Trump initially gained prominence in the 1970s as a real estate developer, taking over his family’s real estate business, which he renamed The Trump Organization. He expanded the company’s operations to include residential and commercial buildings, hotels, casinos, and golf courses, often branding them with his name.\n\nTrump became a well-known public figure through his ownership of the Miss Universe beauty pageants and his reality television show *The Apprentice* (2004–2015), which popularized his catchphrase, "You\'re fired!" His business ventures, pers

## Dense retrieval example
Let’s take a look at a dense retrieval example by using Cohere to search the
Wikipedia page for the film Interstellar. In this example, we will do the
following:
1. Get the text we want to make searchable and apply some light
processing to chunk it into sentences.
2. Embed the sentences.
3. Build the search index.
4. Search and see the results.

In [4]:
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm

# Paste your API key here. Remember to not share publicly
api_key = 'DuJ7RzyWW1iKXVIgeh6SI7j2xXTQgpiIoexIqbhp'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [5]:
text = """
Interstellar is a 2014 epic science fiction film co-written,
directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain,
Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to
survive, the film follows a group of astronauts who travel
through a wormhole near Saturn in search of a new home for
mankind.
Brothers Christopher and Jonathan Nolan wrote the screenplay,
which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in
Physics[4] Kip Thorne was an executive producer, acted as a
scientific consultant, and wrote a tie-in book, The Science of
Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in
the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in
Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and
the company Double Negative created additional digital effects.
Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock,
expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773
million with subsequent re-releases), making it the tenth-highest
grossing film of 2014.
It received acclaim for its performances, direction, screenplay,
musical score, visual effects, ambition, themes, and emotional
weight.
It has also received praise from many astronomers for its
scientific accuracy and portrayal of theoretical astrophysics.
Since its premiere, Interstellar gained a cult following,[5] and
now is regarded by many sci-fi experts as one of the best
science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy
Awards, winning Best Visual Effects, and received numerous other
accolades"""


# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = [t.strip('\n') for t in texts]

In [6]:
texts

['Interstellar is a 2014 epic science fiction film co-written, \ndirected, and produced by Christopher Nolan',
 ' \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, \nBill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 ' \nSet in a dystopian future where humanity is struggling to \nsurvive, the film follows a group of astronauts who travel \nthrough a wormhole near Saturn in search of a new home for \nmankind',
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, \nwhich had its origins in a script Jonathan developed in 2007',
 ' \nCaltech theoretical physicist and 2017 Nobel laureate in \nPhysics[4] Kip Thorne was an executive producer, acted as a \nscientific consultant, and wrote a tie-in book, The Science of \nInterstellar',
 ' \nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in \nthe Panavision anamorphic format and IMAX 70 mm',
 ' \nPrincipal photography began in late 2013 and took place in \nAlberta, Iceland, and Los Angeles',
 ' \

## Embedding the text chunks
Let’s now embed the texts. We’ll send them to the Cohere API, and get back
a vector for each text:

In [7]:
# Get the embeddings
response = co.embed(
  texts=texts,
  input_type="search_document",
).embeddings

embeds = np.array(response)
print(embeds.shape)

(15, 4096)


### which indicates that we have 15 vectors, each one of size 4,096.

## Building the search index
Before we can search, we need to build a search index. An index stores the
embeddings and is optimized to quickly retrieve the nearest neighbors even
if we have a very large number of points:

In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 95.8 MB/s eta 0:00:00


In [9]:
import faiss
dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
print(index.is_trained)
index.add(np.float32(embeds))


True


## Search the index
We can now search the dataset using any query we want. We simply embed
the query and present its embedding to the index, which will retrieve the
most similar sentence from the Wikipedia article.

In [10]:
def search(query, number_of_results=3):

  # 1. Get the query's embedding
  query_embed = co.embed(texts=[query], input_type="search_query",).embeddings[0]

  # 2. Retrieve the nearest neighbors
  distances , similar_item_ids = index.search(np.float32([query_embed]), number_of_results)

  # 3. Format the results
  texts_np = np.array(texts) # Convert texts list to numpy for easier indexing

  results = pd.DataFrame(data={'texts': texts_np[similar_item_ids[0]], 'distance': distances[0]})

  # 4. Print and return the results
  print(f"Query:'{query}'\n Nearest neighbors:")
  return results

In [11]:
query = "how precise was the science"
results = search(query)
results

Query:'how precise was the science'
 Nearest neighbors:


,texts,distance
0,\nIt has also received praise from many astro...,10267.431641
1,"\nSince its premiere, Interstellar gained a c...",12490.475586
2,\nCaltech theoretical physicist and 2017 Nobe...,12507.566406


In [12]:
!pip install rank_bm25

In [14]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):

  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)
    if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
        tokenized_doc.append(token)
  return tokenized_doc

tokenized_corpus = []
for passage in tqdm(texts):
  tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

def keyword_search(query, top_k=3, num_candidates=15):
  print("Input question:", query)

  ##### BM25 search (lexical search) #####
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))

  top_n = np.argpartition(bm25_scores, -num_candidates)[num_candidates:]

  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]

  bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
  print(f"Top-3 lexical search (BM25) hits")

  for hit in bm25_hits[0:top_k]:
    print("\t{:.3f}\t{}".format(hit['score'],texts[hit['corpus_id']].replace("\n", " ")))

100%|██████████| 15/15 [00:00<00:00, 11418.25it/s]


In [15]:
keyword_search(query = "how precise was the science")

Input question: how precise was the science
Top-3 lexical search (BM25) hits


In [16]:
query = "how precise was the science"
results = co.rerank(query=query, documents=texts, top_n=3,
return_documents=
True
)
results.results

[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text=' \nIt has also received praise from many astronomers for its \nscientific accuracy and portrayal of theoretical astrophysics'), index=12, relevance_score=0.15239799),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text=' \nThe film had a worldwide gross over $677 million (and $773 \nmillion with subsequent re-releases), making it the tenth-highest \ngrossing film of 2014'), index=10, relevance_score=0.050354082),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Interstellar is a 2014 epic science fiction film co-written, \ndirected, and produced by Christopher Nolan'), index=0, relevance_score=0.0350424)]

In [17]:
for idx, result in enumerate(results.results):
  print(idx, result.relevance_score , result.document.text)

0 0.15239799  
It has also received praise from many astronomers for its 
scientific accuracy and portrayal of theoretical astrophysics
1 0.050354082  
The film had a worldwide gross over $677 million (and $773 
million with subsequent re-releases), making it the tenth-highest 
grossing film of 2014
2 0.0350424 Interstellar is a 2014 epic science fiction film co-written, 
directed, and produced by Christopher Nolan


In [19]:
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    print(f"Top-3 lexical search (BM25) hits")

    for hit in bm25_hits[0:top_k]:
      print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))
      #Add re-ranking
      docs = [texts[hit['corpus_id']] for hit in bm25_hits]
      print(f"\nTop-3 hits by rank-API ({len(bm25_hits)} BM25 hitsre-ranked)")
      results = co.rerank(query=query, documents=docs, top_n=top_k, return_documents=True)
      # print(results.results)
    for hit in results.results:
      # print(hit)
        print("\t{:.3f}\t{}".format(hit.relevance_score, hit.document.text.replace("\n", " ")))

In [20]:
keyword_and_reranking_search(query = "how precise was the science")

Input question: how precise was the science
Top-3 lexical search (BM25) hits
	1.789	Interstellar is a 2014 epic science fiction film co-written,  directed, and produced by Christopher Nolan

Top-3 hits by rank-API (5 BM25 hitsre-ranked)
	1.373	  Caltech theoretical physicist and 2017 Nobel laureate in  Physics[4] Kip Thorne was an executive producer, acted as a  scientific consultant, and wrote a tie-in book, The Science of  Interstellar

Top-3 hits by rank-API (5 BM25 hitsre-ranked)
	0.000	  The film had a worldwide gross over $677 million (and $773  million with subsequent re-releases), making it the tenth-highest  grossing film of 2014

Top-3 hits by rank-API (5 BM25 hitsre-ranked)
	0.050	  The film had a worldwide gross over $677 million (and $773  million with subsequent re-releases), making it the tenth-highest  grossing film of 2014
	0.035	Interstellar is a 2014 epic science fiction film co-written,  directed, and produced by Christopher Nolan
	0.031	  Caltech theoretical physic